In [1]:
import os
os.chdir('../')

## Imports

In [2]:
import pandas as pd
import datetime as dt

from risk_suite.economics import EconomicsCalculator

## Data Collect

#### Score Imobiliária

In [3]:
all_preds = []

for month in range(1, 7):
    all_preds.append(f"notebooks/outputs/df_preds_20220{month}.csv")

df_preds = (
    pd.concat([pd.read_csv(f) for f in all_preds ])
    [["dt_calendar", "id_imobiliaria", "segments"]]
    .rename(columns={"dt_calendar": "dt_ativacao"})
)

df_preds["dt_ativacao"] = pd.to_datetime(df_preds["dt_ativacao"]).dt.strftime("%Y-%m")

In [4]:
info_contracts = (
    pd.read_csv("data/info_contracts.csv")[["dt_ativacao", "id_imobiliaria", "id_contrato"]]
    .rename(columns={"id_contrato": "contract_id"})
)

info_contracts["dt_ativacao"] = (pd.to_datetime(info_contracts["dt_ativacao"]).dt.strftime("%Y-%m"))

In [5]:
df = info_contracts.merge(df_preds, on=["dt_ativacao", "id_imobiliaria"])
df

,dt_ativacao,id_imobiliaria,contract_id,segments
0,2022-01,38,472480,B
1,2022-01,38,473211,B
2,2022-01,38,477113,B
3,2022-01,38,479053,B
4,2022-01,38,479355,B
...,...,...,...,...
35243,2022-05,28927,617454,A
35244,2022-06,28927,641306,A
35245,2022-06,28927,650933,A
35246,2022-06,28927,657806,A


#### Calculadora

In [6]:
contracts = pd.read_parquet('../risk_suite/data/contracts.parquet')
defaults = pd.read_parquet('../risk_suite/data/defaults.parquet')
recoveries = pd.read_parquet('../risk_suite/data/recoveries.parquet')
revenues = pd.read_parquet('../risk_suite/data/revenues.parquet')

base_features = [
    'contract_id',
    'activation_date',
    'churn_date',
    'activation_month',
    'activation_quarter',
    'score_serasa',
    'rating',
    'rental_value',
]

contracts = contracts[base_features]
contracts = contracts.dropna(subset=['activation_date', 'rating'])
contracts = contracts.loc[lambda x: x['activation_quarter'] >= pd.Period('2020Q1')]
contracts = contracts.replace(['E1', 'E2', 'E3'], 'E')

## Auxiliary Functions

In [7]:
def report_economics2(aggkeys, revenues_pivot, defaults_pivot, recoveries_pivot):

    economics_df = pd.DataFrame(
        {
            'n_contracts': revenues_pivot.groupby(aggkeys).size(),
            'revenue_value': (revenues_pivot.sum(axis=1)).groupby(aggkeys).mean(),
            'prob_default': (defaults_pivot.sum(axis=1) > 0).groupby(aggkeys).mean(),
            'default_value': (defaults_pivot.sum(axis=1)).groupby(aggkeys).mean(),
            'recovery_value': (recoveries_pivot.sum(axis=1)).groupby(aggkeys).mean()
        }
    )

    economics_df = (
        economics_df
        .assign(recovery_efficiency=lambda x: x['recovery_value'] / x['default_value'])
        .assign(unit_economics=lambda x: x['revenue_value'] - x['default_value'] + x['recovery_value'])
        .assign(aggregate_margin=lambda x: x['unit_economics'] * x['n_contracts'])
    )

    return economics_df.sort_index(ascending=[True, False])

## Economics report

In [8]:
df = contracts.merge(df, on="contract_id")

In [9]:
econ_calculator = EconomicsCalculator(df, defaults, recoveries, revenues, max_history_date='2023-01')

defaults_pivot = econ_calculator._build_defaults_pivot()
recoveries_pivot = econ_calculator._build_recoveries_pivot()
revenues_pivot = econ_calculator._build_revenues_pivot()

report = report_economics2(["segments"], revenues_pivot, defaults_pivot, recoveries_pivot)

/Users/raquel.camara/opt/anaconda3/lib/python3.9/site-packages/risk_suite/economics.py:139: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  event_data


In [10]:
report

,n_contracts,revenue_value,prob_default,default_value,recovery_value,recovery_efficiency,unit_economics,aggregate_margin
segments,,,,,,,,
A,1271,1335.677663,0.052714,270.501857,98.335980,0.363532,1163.511786,1478823.48
B,3354,1214.062275,0.064103,347.241855,107.349120,0.309148,974.169541,3267364.64
C,5833,1140.376235,0.065832,353.497725,107.667615,0.304578,894.546125,5217887.55
D,5967,1132.072073,0.108765,696.862427,195.004444,0.279832,630.214091,3760487.48
E,18454,1136.454054,0.124688,870.064657,239.138413,0.274851,505.527810,9329010.21
